# STRAVA API AND CYCLING ACTIVITY SUMMARY DATA
---
---


In [1]:
# https://towardsdatascience.com/using-the-strava-api-and-pandas-to-explore-your-activity-data-d94901d9bfde
# https://www.youtube.com/watch?v=sgscChKfGyg&t=258s
# https://github.com/franchyze923/Code_From_Tutorials/blob/master/Strava_Api/strava_api.py
# Lesson 5.01 APIs
# Lesson 8.02 Timeseries

# Imports
---

In [2]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests
import urllib3

%matplotlib inline
pd.set_option('display.max_columns', None)

# API
---

In [3]:
client_id = "89682"
secret = '0211f1fcd1e21bf52f9690ed169fcbd4d886f602'
refresh_token = '7a3f31cb96a242deb9e08de21eee388496d44bd0'

In [4]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': client_id,
    'client_secret': secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print(f'Status: {res.status_code}')

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

Requesting Token...

Status: 200


In [5]:
# JSON
activities = json_normalize(my_dataset)

In [6]:
# activities.head()

In [7]:
activities.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'id', 'start_date',
       'start_date_local', 'timezone', 'utc_offset', 'location_city',
       'location_state', 'location_country', 'achievement_count',
       'kudos_count', 'comment_count', 'athlete_count', 'photo_count',
       'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged',
       'gear_id', 'start_latlng', 'end_latlng', 'average_speed', 'max_speed',
       'has_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option',
       'upload_id', 'upload_id_str', 'external_id', 'from_accepted_tag',
       'pr_count', 'total_photo_count', 'has_kudoed', 'athlete.id',
       'athlete.resource_state', 'map.id', 'map.summary_polyline',
       'map.resource_state', 'workout_type', 'average_cadence',
       'average_heartrate', 'max_heartrate', 'elev_high', 'elev_low',
       'suffer_score', 'average_watts', 'max_watts', 'weighted_average_

In [8]:
#Break date into start time and date
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date

In [9]:
# clean up to wanted columns
cols = ['id', 'type', 'start_date_local', 'start_time', 'distance', 'moving_time', 'elapsed_time', 
        'average_speed', 'max_speed', 'average_cadence', 'average_heartrate', 
        'max_heartrate', 'suffer_score', 'total_elevation_gain', 'elev_high', 
        'elev_low']
activities = activities[cols]
activities = activities[activities['type'] == 'Ride']

In [10]:
activities.head()

,id,type,start_date_local,start_time,distance,moving_time,elapsed_time,average_speed,max_speed,average_cadence,average_heartrate,max_heartrate,suffer_score,total_elevation_gain,elev_high,elev_low
3,7624700027,Ride,2022-08-11,15:20:11,26598.9,4575,4750,5.814,8.734,64.5,127.7,158.0,30.0,54.0,38.4,15.8
4,7611476714,Ride,2022-08-09,12:33:34,35604.4,6523,7927,5.458,8.480,61.9,128.0,156.0,44.0,101.0,38.4,8.4
5,7610864586,Ride,2022-08-09,11:52:46,4376.1,1559,1559,2.807,5.350,30.5,84.2,98.0,2.0,11.0,35.0,19.0
6,7599904305,Ride,2022-08-07,11:42:57,4563.0,715,779,6.382,7.659,73.5,131.1,151.0,5.0,4.0,36.2,28.4
7,7594679706,Ride,2022-08-06,13:31:46,50172.3,7711,9425,6.507,10.842,69.1,152.8,173.0,204.0,125.0,49.4,13.2


In [11]:
activities.shape

(96, 16)

In [14]:
# activities.to_csv('../data/strava.csv', index = False)